# Stage 3 - Enitity Matching Notebook
- Ankit Vij
- Amanpreet Singh Saini
- Joel Haynie

Date: 2018-04-18

Stages of our Pipeline: Clean -> Block -> Label -> ML'ing a Matcher

# Clean
We executed ```prepTableA.py``` & ```prepTableB.py```.

These take the raw two tables from Stage # 2 to a cleaned Table A & B

These go Column by Column and clean up and transform the data into a consistant and usable form.  Defaults were also defined.

 # Blocking
 We executed the ```blocker.py``` below is a summary.

In [36]:
import py_entitymatching as em
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [37]:
# Load csv files as dataframes and set the key attribute in the dataframe
path_A = '..'+ os.sep + 'data' + os.sep + 'A.csv'
A = em.read_csv_metadata(path_A, key='ID')
path_B = '..'+ os.sep + 'data' + os.sep + 'B.csv'
B = em.read_csv_metadata(path_B, key='ID')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [38]:
#Number of Tuples in A
print("A Tuples: " + str(len(A)))
A.head(3)

A Tuples: 3250


,ID,Title,Score,Rating,Genre,Directed By,Written By,Box Office,Release Date,Runtime,Studio
0,A_0001,Avengers: Infinity War,0,NOT RATED,Fantasy,Anthony Russo,Christopher Markus,0,2018-04-27,156,Marvel Studios
1,A_0002,Tomb Raider,69,PG-13,Action,Roar Uthaug,Geneva Robertson-Dworet,31318108,2018-03-16,118,GK Films
2,A_0003,Black Panther,78,PG-13,Sci-Fi,Ryan Coogler,Ryan Coogler,614258236,2018-02-16,134,Marvel Studios


In [39]:
#Number of Tuples in A
print("A Tuples: " + str(len(B)))
B.head(3)

A Tuples: 3005


,ID,Title,Score,Rating,Genre,Directed By,Written By,Box Office,Release Date,Runtime,Studio
0,B_0001,Justice League,40,PG-13,Drama,Zack Snyder,Chris Terrio,227032490,2017-11-17,110,Warner Bros. Pictures
1,B_0002,Ferdinand,71,PG,Drama,Carlos Saldanha,Robert L. Baird,70466891,2017-12-15,0,NaN
2,B_0003,Court,100,NOT RATED,Drama,Chaitanya Tamhane,Chaitanya Tamhane,0,2015-07-15,116,Zeitgeist Films


In [40]:
print("Possible Tuple Matches: " + str(len(A)*len(B)))

Possible Tuple Matches: 9766250


Gernerate 3 candidate sets:
1. AttrEquivalenceBlocker on the movie 'Release Date'
1. OverlapBlocker on the movie 'Title'
1. AttrEquivalenceBlocker on the movie 'Title'.

These are all Unioned together into set D
Finally a Rule Based Blocker over the Levenshtein distance for the 'Title' < 0.4


In [41]:
#1 
ab = em.AttrEquivalenceBlocker()
# Block tables using 'year' attribute : same year include in candidate set
C1 = ab.block_tables(A, B, 'Release Date', 'Release Date',
                     l_output_attrs=['Title', 'Genre', 'Score', 'Release Date', 'Rating', 'Directed By', 'Written By', 'Studio'],
                     r_output_attrs=['Title', 'Genre', 'Score', 'Release Date', 'Rating', 'Directed By', 'Written By', 'Studio'])
print("C1 Set Size: ", len(C1))

C1 Set Size:  191237


**Debugging C1**- 
We observed the debug blocker E1 which found very few matches, hence signifying that we have covered all of the true matches. But the size of C1 is still 191237, as noted above, which is significantly higher in proportion to the true matches which are 607. Therefore, we decided to apply more blockers in this scheme to do more aggressive blocking. 

In [63]:
E1 = em.debug_blocker(C1, A, B, output_size=200)
E1.head()

,_id,ltable_ID,rtable_ID,ltable_Title,ltable_Rating,ltable_Genre,ltable_Directed By,ltable_Written By,ltable_Release Date,ltable_Studio,rtable_Title,rtable_Rating,rtable_Genre,rtable_Directed By,rtable_Written By,rtable_Release Date,rtable_Studio
0,0,A_2003,B_1318,Lady in the Water,PG-13,Fantasy,M. Night Shyamalan,M. Night Shyamalan,2006-07-21,Warner Bros.,The Visit,PG-13,Thriller,M. Night Shyamalan,M. Night Shyamalan,2015-09-11,Universal Pictures
1,1,A_2911,B_0524,Resident Evil: Retribution,R,Horror,Paul W.S. Anderson,Paul W.S. Anderson,2012-09-14,Constantin Film International,Resident Evil: The Final Chapter,R,Action,Paul W.S. Anderson,Paul W.S. Anderson,2017-01-27,Sony Pictures
2,2,A_0543,B_0178,Kingsman: The Secret Service,R,Comedy,Matthew Vaughn,Jane Goldman,2015-02-13,Twentieth Century Fox,Kingsman: The Golden Circle,R,Adventure,Matthew Vaughn,Jane Goldman,2017-09-22,20th Century Fox
3,3,A_2688,B_0524,Resident Evil: Afterlife,R,Horror,Paul W.S. Anderson,Paul W.S. Anderson,2010-09-10,Constantin Film,Resident Evil: The Final Chapter,R,Action,Paul W.S. Anderson,Paul W.S. Anderson,2017-01-27,Sony Pictures
4,4,A_2670,B_1318,The Last Airbender,PG,Family,M. Night Shyamalan,M. Night Shyamalan,2010-07-01,Paramount Pictures,The Visit,PG-13,Thriller,M. Night Shyamalan,M. Night Shyamalan,2015-09-11,Universal Pictures


In [42]:
#2
ob = em.OverlapBlocker()
# Block over title attribute
C2 = ob.block_tables(A, B, 'Title', 'Title', show_progress=False, overlap_size=2, rem_stop_words=True,
                     l_output_attrs=['Title', 'Genre', 'Score', 'Release Date', 'Rating', 'Directed By', 'Written By','Studio'],
                     r_output_attrs=['Title', 'Genre', 'Score', 'Release Date', 'Rating', 'Directed By', 'Written By','Studio'])
print("C2 Set Size: ", len(C2))

C2 Set Size:  1826


**Debugging C2** - 
On applying this second blocker and debuggging it, the debugger returned a lot of possible matches that might have been removed during this blocking step, hence making this step very aggressive. So, we decided to merge (union) the results of both of these blockers.

In [64]:
E2 = em.debug_blocker(C2, A, B, output_size=200)
E2.head()

,_id,ltable_ID,rtable_ID,ltable_Title,ltable_Rating,ltable_Genre,ltable_Directed By,ltable_Written By,ltable_Release Date,ltable_Studio,rtable_Title,rtable_Rating,rtable_Genre,rtable_Directed By,rtable_Written By,rtable_Release Date,rtable_Studio
0,0,A_0876,B_0991,Allegiant,PG-13,Mystery,Robert Schwentke,Noah Oppenheim,2016-03-18,Mandeville Films,The Divergent Series: Allegiant,PG-13,Sci-Fi,Robert Schwentke,Noah Oppenheim,2016-03-18,Lionsgate
1,1,A_0874,B_1318,The Sixth Sense,PG-13,Drama,M. Night Shyamalan,M. Night Shyamalan,1999-08-06,Hollywood Pictures,The Visit,PG-13,Thriller,M. Night Shyamalan,M. Night Shyamalan,2015-09-11,Universal Pictures
2,2,A_1687,B_1318,The Village,PG-13,Drama,M. Night Shyamalan,M. Night Shyamalan,2004-07-30,Touchstone Pictures,The Visit,PG-13,Thriller,M. Night Shyamalan,M. Night Shyamalan,2015-09-11,Universal Pictures
3,3,A_1124,B_0560,Unbreakable,PG-13,Sci-Fi,M. Night Shyamalan,M. Night Shyamalan,2000-11-22,Touchstone Pictures,Split,PG-13,Drama,M. Night Shyamalan,M. Night Shyamalan,2017-01-20,Universal Pictures
4,4,A_0543,B_0178,Kingsman: The Secret Service,R,Comedy,Matthew Vaughn,Jane Goldman,2015-02-13,Twentieth Century Fox,Kingsman: The Golden Circle,R,Adventure,Matthew Vaughn,Jane Goldman,2017-09-22,20th Century Fox


In [43]:
#3
# Attribute Equivalence Blocker for Title
C3 = ab.block_tables(A, B, 'Title', 'Title',
                     l_output_attrs=['Title', 'Genre', 'Score', 'Release Date', 'Rating', 'Directed By', 'Written By','Studio'],
                     r_output_attrs=['Title', 'Genre', 'Score', 'Release Date', 'Rating', 'Directed By', 'Written By','Studio'])
print("C3 Set Size: ", len(C3))

C3 Set Size:  607


**Debugging C3** - 
Since the last blocker was missing out on a good fraction of the matches, we decided to add an additional blocking step which would improve the coverage of the true matches, if any of them were left out during the previous two blocking steps. The debugger after this step doesn't show a lot of possible candidates that we might have missed.

In [65]:
E3 = em.debug_blocker(C3, A, B, output_size=200)
E3.head()

,_id,ltable_ID,rtable_ID,ltable_Title,ltable_Rating,ltable_Genre,ltable_Directed By,ltable_Written By,ltable_Release Date,ltable_Studio,rtable_Title,rtable_Rating,rtable_Genre,rtable_Directed By,rtable_Written By,rtable_Release Date,rtable_Studio
0,0,A_1154,B_1890,The Hunger Games: Mockingjay - Part 2,PG-13,Sci-Fi,Francis Lawrence,Peter Craig,2015-11-20,Color Force,The Hunger Games: Mockingjay - Part 1,PG-13,Drama,Francis Lawrence,Danny Strong,2014-11-21,Lionsgate Films
1,1,A_0543,B_0178,Kingsman: The Secret Service,R,Comedy,Matthew Vaughn,Jane Goldman,2015-02-13,Twentieth Century Fox,Kingsman: The Golden Circle,R,Adventure,Matthew Vaughn,Jane Goldman,2017-09-22,20th Century Fox
2,2,A_0977,B_0560,The Visit,PG-13,Thriller,M. Night Shyamalan,M. Night Shyamalan,2015-09-11,Blinding Edge Pictures,Split,PG-13,Drama,M. Night Shyamalan,M. Night Shyamalan,2017-01-20,Universal Pictures
3,3,A_0897,B_0779,E.T. the Extra-Terrestrial,PG,Family,Steven Spielberg,Melissa Mathison,1982-06-11,Universal Pictures,The BFG,PG,Sci-Fi,Steven Spielberg,Melissa Mathison,2016-07-01,Walt Disney Pictures
4,4,A_1166,B_1180,The Hunger Games: Mockingjay - Part 1,PG-13,Sci-Fi,Francis Lawrence,Peter Craig,2014-11-21,Color Force,The Hunger Games: Mockingjay - Part 2,PG-13,Drama,Francis Lawrence,Danny Strong,2015-11-20,Lionsgate Films


In [44]:
#Union the last three
D = em.combine_blocker_outputs_via_union([C1, C2, C3])
print("D Set Size: ", len(D))

D Set Size:  192807


In [45]:
# Rule based blocker on D
block_f = em.get_features_for_blocking(A, B, validate_inferred_attr_types=False)
rb = em.RuleBasedBlocker()
rb.add_rule(['Title_Title_lev_sim(ltuple, rtuple) < 0.4'], block_f)
C = rb.block_candset(D, show_progress=False)
print("C Set Size: ", len(C))

C Set Size:  3335


**Debugging C** -
The debugger shows only a few candidate matches that we might have missed. We take a look on the debugger output and saw that the listed candidates were infact different real world entities. They were only shown as a possible match by the debugger possibbly due to the string similarity. Thus, we conclude that as the effective number of matches shown by the debugger output is negligible, as well as the size of the candidate set is not skewed in terms of true vs non-true matches, we decided on this final blocking strategy and move on the matching. 

In [66]:
E = em.debug_blocker(C, A, B, output_size=200)
E.head()

,_id,ltable_ID,rtable_ID,ltable_Title,ltable_Rating,ltable_Genre,ltable_Directed By,ltable_Written By,ltable_Release Date,ltable_Studio,rtable_Title,rtable_Rating,rtable_Genre,rtable_Directed By,rtable_Written By,rtable_Release Date,rtable_Studio
0,0,A_0582,B_1308,Back to the Future,PG,Sci-Fi,Robert Zemeckis,Robert Zemeckis,1985-07-03,Universal Pictures,The Walk,PG,Drama,Robert Zemeckis,Robert Zemeckis,2015-10-09,Sony Pictures
1,1,A_2127,B_2057,Blair Witch,R,Thriller,Adam Wingard,Simon Barrett,2016-09-16,Lionsgate,The Guest,R,Thriller,Adam Wingard,Simon Barrett,2014-09-17,Picturehouse Films
2,2,A_2701,B_0218,The Osiris Child,NOT RATED,Drama,Shane Abbess,Shane Abbess,2017-05-18,Storm Vision Entertainment,The Osiris Child: Science Fiction Volume One,NOT RATED,Drama,Shane Abbess,Shane Abbess,2017-10-06,NaN
3,3,A_0977,B_0560,The Visit,PG-13,Thriller,M. Night Shyamalan,M. Night Shyamalan,2015-09-11,Blinding Edge Pictures,Split,PG-13,Drama,M. Night Shyamalan,M. Night Shyamalan,2017-01-20,Universal Pictures
4,4,A_1227,B_0020,The Devil's Backbone,R,Drama,Guillermo del Toro,Guillermo del Toro,2001-04-20,"Deseo, El",The Shape of Water,R,Drama,Guillermo del Toro,Guillermo del Toro,2017-12-22,Fox Searchlight Pictures


We Saved off C set to a csv file in the data directory.

We generated our set S with ```S = em.sample_table(C, 300)```

We Then labeled our data with the ```G = em.label_table(S, 'label')```

And finally saved G set to a csv file in the data directory.

# ML'ing a Matcher
We executed ```createClassifier.py``` below is a summary.

In [67]:
path_G = '..' + os.sep + 'data' + os.sep + 'G.csv'
G = em.read_csv_metadata(path_G, key='_id', ltable=A, rtable=B, fk_ltable='ltable_ID', fk_rtable='rtable_ID')
print("G Tuples: "+str(len(G)))
G.head()

Metadata file is not present in the given path; proceeding to read the csv file.


G Tuples: 300


,Unnamed: 0,_id,ltable_ID,rtable_ID,ltable_Title,ltable_Genre,ltable_Score,ltable_Release Date,ltable_Rating,ltable_Directed By,...,ltable_Studio,rtable_Title,rtable_Genre,rtable_Score,rtable_Release Date,rtable_Rating,rtable_Directed By,rtable_Written By,rtable_Studio,label
0,783,783,A_0012,B_0704,Justice League,Fantasy,68,2017-11-17,PG-13,Zack Snyder,...,Atlas Entertainment,Justice League Dark,Action,71,1900-1-1,R,Jay Oliva,Ernie Altbacker,NaN,0
1,1199,1199,A_0016,B_0395,The Flash,Drama,80,1900-1-1,TV-PG,NaN,...,Berlanti Productions,The Sea,Drama,47,1900-1-1,NOT RATED,Stephen Brown,John Banville,NaN,0
2,1332,1332,A_0016,B_2592,The Flash,Drama,80,1900-1-1,TV-PG,NaN,...,Berlanti Productions,The Patrol,Drama,64,1900-1-1,NOT RATED,Tom Petch,Tom Petch,Kasbah-Film Tanger,0
3,4104,4104,A_0046,B_1102,Prison Break,Drama,84,1900-1-1,TV-14,NaN,...,Adelstein-Parouse Productions,NaN,NaN,0,1900-1-1,NOT RATED,NaN,NaN,NaN,0
4,4361,4361,A_0047,B_2257,Ash vs Evil Dead,Fantasy,85,1900-1-1,TV-MA,NaN,...,Renaissance Pictures,See No Evil 2,Horror,60,1900-1-1,R,Jen Soska,Bobby Lee Darby,Lionsgate Films,0


In [47]:
# Create our Sets I & J
IJ = em.split_train_test(G, train_proportion=0.7, random_state=0)
I = IJ['train']
J = IJ['test']
print("I Tuples: "+str(len(I)))
print("J Tuples: "+str(len(J)))

I Tuples: 210
J Tuples: 90


In [48]:
# Obtain our feature sets
feature_table = em.get_features_for_matching(A, B, validate_inferred_attr_types=True)

# extract our feature vectors from I
H = em.extract_feature_vecs(I, feature_table=feature_table, attrs_after='label', show_progress=False)

# Clean up some missing data.
H.fillna(value=0, inplace=True)

The table shows the corresponding attributes along with their respective types.
Please confirm that the information  has been correctly inferred.
If you would like to skip this validation process in the future,
please set the flag validate_inferred_attr_types equal to false.


,Left Attribute,Right Attribute,Left Attribute Type,Right Attribute Type,Example Features
0,ID,ID,short string (1 word),short string (1 word),Levenshtein Distance; Levenshtein Similarity
1,Title,Title,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
2,Score,Score,numeric,numeric,Exact Match; Absolute Norm
3,Rating,Rating,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
4,Genre,Genre,short string (1 word),short string (1 word),Levenshtein Distance; Levenshtein Similarity
5,Directed By,Directed By,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
6,Written By,Written By,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
7,Box Office,Box Office,numeric,numeric,Exact Match; Absolute Norm
8,Release Date,Release Date,short string (1 word),short string (1 word),Levenshtein Distance; Levenshtein Similarity
9,Runtime,Runtime,numeric,numeric,Exact Match; Absolute Norm


Do you want to proceed? (y/n):y


We Saved off I & J Sets to a csv file in the data directory

We ran through the following classifers using(```em.select_matcher```) with precision and recall on set H (from I):
1. ```em.DTMatcher(name='DecisionTree', random_state=0)```
1. ```em.SVMMatcher(name='SVM', kernel='linear', random_state=0)```
1. ```em.RFMatcher(name='RF', random_state=0)```
1. ```em.LogRegMatcher(name='LogReg', random_state=0)```
1. ```em.LinRegMatcher(name='LinReg')```
1. ```em.NBMatcher(name='NaiveBayes')```

We choose **Random Forest** as it has the highest F1.

In [49]:
# Define our matcher
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', kernel='linear', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

In [50]:
# precision
result = em.select_matcher([dt, svm, rf, lg, ln, nb], table=H,
                           exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
                           k=5,
                           target_attr='label', metric_to_select_matcher='precision', random_state=0)

# recall
result = em.select_matcher([dt, svm, rf, lg, ln, nb], table=H,
                           exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
                           k=5,
                           target_attr='label', metric_to_select_matcher='recall', random_state=0)
print(result['cv_stats'])

        Matcher  Average precision  Average recall  Average f1
0  DecisionTree           0.966667        0.966667    0.966667
1           SVM           0.971429        0.971429    0.969231
2            RF           1.000000        0.966667    0.981818
3        LogReg           0.950000        1.000000    0.971429
4        LinReg           1.000000        0.869524    0.926263
5    NaiveBayes           1.000000        0.926667    0.959596


In [55]:
# Build up or test set feature vectors from J
L = em.extract_feature_vecs(J, feature_table=feature_table, attrs_after='label', show_progress=False)

# Clean up some missing data.
L.fillna(value=0, inplace=True)

#Fit to our set Development Set H (from I)
rf.fit(table=H, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], target_attr='label')

# Use our test set L (from J) to measure the Matchers effectiveness
predictions = rf.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
                         append=True, target_attr='predicted', inplace=False)

# How did it do?
eval_result = em.eval_matches(predictions, 'label', 'predicted')
print('\nRandom Forest Result-')
em.print_eval_summary(eval_result)


Random Forest Result-
Precision : 100.0% (22/22)
Recall : 100.0% (22/22)
F1 : 100.0%
False positives : 0 (out of 22 positive predictions)
False negatives : 0 (out of 68 negative predictions)


**Calculating P/R/F1 on set J for the other matchers**

In [56]:
dt.fit(table=H,
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
       target_attr='label')

predictions = dt.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
                         append=True, target_attr='predicted', inplace=False)

eval_result = em.eval_matches(predictions, 'label', 'predicted')
print('\nDecision Tree Result-')
em.print_eval_summary(eval_result)


Decision Tree Result-
Precision : 91.67% (22/24)
Recall : 100.0% (22/22)
F1 : 95.65%
False positives : 2 (out of 24 positive predictions)
False negatives : 0 (out of 66 negative predictions)


In [57]:
svm.fit(table=H,
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
       target_attr='label')

predictions = svm.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
                         append=True, target_attr='predicted', inplace=False)

eval_result = em.eval_matches(predictions, 'label', 'predicted')
print('\nSVM Result-')
em.print_eval_summary(eval_result)


SVM Result-
Precision : 87.5% (21/24)
Recall : 95.45% (21/22)
F1 : 91.3%
False positives : 3 (out of 24 positive predictions)
False negatives : 1 (out of 66 negative predictions)


In [58]:
lg.fit(table=H,
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
       target_attr='label')

predictions = lg.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
                         append=True, target_attr='predicted', inplace=False)

eval_result = em.eval_matches(predictions, 'label', 'predicted')
print('\nLogistic Reg Result-')
em.print_eval_summary(eval_result)


Logistic Reg Result-
Precision : 88.0% (22/25)
Recall : 100.0% (22/22)
F1 : 93.62%
False positives : 3 (out of 25 positive predictions)
False negatives : 0 (out of 65 negative predictions)


In [60]:
ln.fit(table=H,
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
       target_attr='label')

predictions = ln.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
                         append=True, target_attr='predicted', inplace=False)

eval_result = em.eval_matches(predictions, 'label', 'predicted')
print('\nLinear Regression Result-')
em.print_eval_summary(eval_result)


Linear Regression Result-
Precision : 100.0% (22/22)
Recall : 100.0% (22/22)
F1 : 100.0%
False positives : 0 (out of 22 positive predictions)
False negatives : 0 (out of 68 negative predictions)


In [61]:
nb.fit(table=H,
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
       target_attr='label')

predictions = nb.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
                         append=True, target_attr='predicted', inplace=False)

eval_result = em.eval_matches(predictions, 'label', 'predicted')
print('\nNaive Bayes Result-')
em.print_eval_summary(eval_result)


Naive Bayes Result-
Precision : 91.67% (22/24)
Recall : 100.0% (22/22)
F1 : 95.65%
False positives : 2 (out of 24 positive predictions)
False negatives : 0 (out of 66 negative predictions)


**Fin!**